Initial Data Exploration

In [1]:
#Import dependencies

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# Read in CSV
raw_data_2022_mn_pm25 = "Data/2022_MN_PM_25.csv"
df_raw_2022_mn_pm25 = pd.read_csv(raw_data_2022_mn_pm25)

In [3]:
# Drop unneeded columns
df_raw_2022_mn_pm25_dropped = df_raw_2022_mn_pm25.drop(columns = ["Source", "Site ID", "DAILY_OBS_COUNT", "PERCENT_COMPLETE", 
                                                                  "AQS_PARAMETER_CODE", "AQS_PARAMETER_DESC", "CBSA_CODE", 
                                                                  "CBSA_NAME", "COUNTY_CODE", "STATE_CODE", "STATE"])
#df_raw_2022_mn_pm25_dropped

In [4]:
# Rename columns
columns = {"Date": "Date", "POC": "POC", "Daily Mean PM2.5 Concentration": "Day_Mean_PM25_Conc", "UNITS": "Units", 
           "DAILY_AQI_VALUE":"Daily_AQI", "Site Name": "Site_Name", "COUNTY": "County", "SITE_LATITUDE": "Latitude",
          "SITE_LONGITUDE": "Longitude"}

df_raw_2022_mn_pm25_rename = df_raw_2022_mn_pm25_dropped.rename(columns = columns)

In [5]:
# Add column "AQI_Concern" to reflect AQI category names
df_raw_2022_mn_pm25_rename["AQI_Concern"] = np.where(df_raw_2022_mn_pm25_rename["Daily_AQI"] <= 50, "Good", 
                                                     np.where(df_raw_2022_mn_pm25_rename["Daily_AQI"] <= 100, "Moderate", 
                                                              np.where(df_raw_2022_mn_pm25_rename["Daily_AQI"] <= 150, "Unhealthy for Sensitive Groups", 
                                                                       np.where(df_raw_2022_mn_pm25_rename["Daily_AQI"] <= 200, "Unhealthy", 
                                                                                np.where(df_raw_2022_mn_pm25_rename["Daily_AQI"] <= 300, "Very Unhealthy", "Hazardous")))))

In [6]:
# Change Date column to datetime
df_raw_2022_mn_pm25_rename["Date"] = pd.to_datetime(df_raw_2022_mn_pm25_rename["Date"])

In [18]:
# Generalize processing raw CSV

def clean_data(raw_csv_path):
    # Clear variables
    column_names = []
    
    # Read in raw data
    data_path = raw_csv_path
    raw_dataframe = pd.read_csv(raw_csv_path)
    
    # Drop unnecessary columns
    raw_df_dropped = raw_dataframe.drop(columns = ["Source", "Site ID", "DAILY_OBS_COUNT", "PERCENT_COMPLETE", 
                                                   "AQS_PARAMETER_CODE", "AQS_PARAMETER_DESC", "CBSA_CODE", 
                                                   "CBSA_NAME", "COUNTY_CODE", "STATE_CODE", "STATE"])
    
    # Rename columns
    column_names = {"Date": "Date", "POC": "POC", "Daily Mean PM2.5 Concentration": "Day_Mean_PM25_Conc", "UNITS": "Units", 
           "DAILY_AQI_VALUE":"Daily_AQI", "Site Name": "Site_Name", "COUNTY": "County", "SITE_LATITUDE": "Latitude",
          "SITE_LONGITUDE": "Longitude"}
    
    raw_df_renamed = raw_df_dropped.rename(columns = column_names)
    
    # Add "AQI_Concern" column to reflect AQI category names
    raw_df_renamed["AQI_Concern"] = np.where(raw_df_renamed["Daily_AQI"] <= 50, "Good", 
                                                     np.where(raw_df_renamed["Daily_AQI"] <= 100, "Moderate", 
                                                              np.where(raw_df_renamed["Daily_AQI"] <= 150, "Unhealthy for Sensitive Groups", 
                                                                       np.where(raw_df_renamed["Daily_AQI"] <= 200, "Unhealthy", 
                                                                                np.where(raw_df_renamed["Daily_AQI"] <= 300, "Very Unhealthy", "Hazardous")))))
    
    return raw_df_renamed 

In [19]:
### TEST FUNCTION!!! ###

clean_data("Data/2021_MN_PM25.csv")


,Date,POC,Day_Mean_PM25_Conc,Units,Daily_AQI,Site_Name,County,Latitude,Longitude,AQI_Concern
0,01/01/2021,1,18.0,ug/m3 LC,63,Anoka County Airport,Anoka,45.13768,-93.207615,Moderate
1,01/07/2021,1,24.9,ug/m3 LC,78,Anoka County Airport,Anoka,45.13768,-93.207615,Moderate
2,01/10/2021,1,16.4,ug/m3 LC,60,Anoka County Airport,Anoka,45.13768,-93.207615,Moderate
3,01/11/2021,1,7.3,ug/m3 LC,30,Anoka County Airport,Anoka,45.13768,-93.207615,Good
4,01/13/2021,1,14.9,ug/m3 LC,57,Anoka County Airport,Anoka,45.13768,-93.207615,Moderate
...,...,...,...,...,...,...,...,...,...,...
8537,12/27/2021,3,5.0,ug/m3 LC,21,St. Michael Elementary School,Wright,45.20916,-93.669210,Good
8538,12/28/2021,3,6.7,ug/m3 LC,28,St. Michael Elementary School,Wright,45.20916,-93.669210,Good
8539,12/29/2021,3,5.4,ug/m3 LC,23,St. Michael Elementary School,Wright,45.20916,-93.669210,Good
8540,12/30/2021,3,11.8,ug/m3 LC,49,St. Michael Elementary School,Wright,45.20916,-93.669210,Good


In [ ]:
# Generalize the color-coded scatter plot process

def scatter_plot_color(df_name, pollutant_name, location_name, year):
    moderate_bound = 50
    unhealthy_for_special_groups_bound = 100
    unhealthy_bound = 150
    very_unhealthy_bound = 200
    hazardous_bound = 300

    x_values = df_name["Date"]
    y_values = df_name["Daily_AQI"]
    
    good = np.ma.masked_where(y > moderate_bound, y)
    moderate = np.ma.masked_where((y <= moderate_bound) | (y > unhealthy_for_special_groups_bound), y)
    unhealthy_special_groups = np.ma.masked_where((y <= unhealthy_for_special_groups_bound) | (y > unhealthy_bound), y)
    unhealthy = np.ma.masked_where((y <= unhealthy_bound) | (y > very_unhealthy_bound), y)
    very_unhealthy = np.ma.masked_where((y <= very_unhealthy_bound) | (y > hazardous_bound), y)
    hazardous = np.ma.masked_where(y <= hazardous_bound, y)
    
    fig, ax = plt.subplots()
    ax.scatter(x, good, color = 'green', marker = '.')
    ax.scatter(x, moderate, color = 'yellow', marker = '.')
    ax.scatter(x, unhealthy_special_groups, color = 'orange', marker = '.')
    ax.scatter(x, unhealthy, color = 'red', marker = '.')
    ax.scatter(x, very_unhealthy, color = 'purple', marker = '.')
    ax.scatter(x, hazardous, color = 'maroon', marker = '.')

    plt.xlabel("Date")
    plt.ylabel("Air Quality Index")
    plt.title(f'{pollutant_name} AQI for {location_name} in {year}')
    
    plt.show()

In [ ]:
# Generalize separating location data and identifying POC with most data

def data_split(raw_df_name, state, pollutant_name, year):
    # Clear variables
    location_list = []
    data_dict = {}
    
    # Create list of unique locations
    location_list = raw_df_name["Site_Name"].unique().tolist()
    for location in location_list:
        location_df = raw_df_name.loc[raw_df_name["Site_Name"] == location]
        poc_list_location = dict(location_df.value_counts("POC"))
    
        # Choose POC with most data
        if len(poc_list_location) > 1:
            location_df = location_df.loc[location_df["POC"] == max(poc_list_location, key = poc_list_location.get)]
    
        location_standard_name = location.replace(' ', '_').replace('/', "_").replace(":", "").replace("-", "").replace(".", "")
        new_name = f"{location_standard_name}_{state}_{pollutant_name}_{year}"
        data_dict[new_name] = location_df
        
    return data_dict